In [1]:
import requests
import json
import csv
import time
import threading

In [2]:
# Define the API keys and CSV file path
api_key = ""
api_secret = ""
csv_file_path = "flight_data.csv"

In [3]:
# Function to get OAuth Token
def get_access_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": api_key,
        "client_secret": api_secret
    }
    response = requests.post(url, data=data, verify=False)  # Disable SSL verification
    token_response = response.json()
    return token_response["access_token"]


In [4]:
# Function to fetch flight destination data
def fetch_and_save_flight_data():
    token = get_access_token()
    headers = {
        "Authorization": f"Bearer {token}"
    }
    api_url = "https://test.api.amadeus.com/v1/shopping/flight-destinations?origin=PAR"
    response = requests.get(api_url, headers=headers, verify=False)  # Disable SSL verification
    flight_data = response.json()
    save_to_csv(flight_data)


In [5]:
# Function to save flight data to CSV
def save_to_csv(flight_response):
    file_exists = False
    try:
        # Check if the CSV file exists
        file_exists = open(csv_file_path).close()
    except:
        pass

    with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["Origin", "Destination", "DepartureDate", "ReturnDate", "Price"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write header only if file is new
        if not file_exists:
            writer.writeheader()

        # Write flight data
        for flight in flight_response["data"]:
            writer.writerow({
                "Origin": flight["origin"],
                "Destination": flight["destination"],
                "DepartureDate": flight["departureDate"],
                "ReturnDate": flight["returnDate"],
                "Price": flight["price"]["total"]
            })

    print(f"Data saved to {csv_file_path} at {time.ctime()}")


In [6]:
# Function to run the data fetching periodically (every 2 minutes)
def start_scheduler(interval_minutes=2):
    while True:
        fetch_and_save_flight_data()
        time.sleep(interval_minutes * 60)  # Sleep for 2 minutes


In [ ]:
# Start the process in a background thread so that it runs every 2 minutes
thread = threading.Thread(target=start_scheduler)
thread.start()

# In Jupyter, this will allow you to stop the background task by interrupting the kernel when needed
print("Fetching flight data every 2 minutes... Press stop (interrupt the kernel) to stop.")